## Template folders and files

- [ ] TODO

## Local Development Install
This cell installs the package in development mode (-e flag).
After changes to source files, restart kernel to see updates.

In [2]:
!pip install -e .

Obtaining file:///Users/lorduk/Documents/Pyperiscope
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pyperiscope (pyproject.toml) ... done
  Created wheel for pyperiscope: filename=pyperiscope-0.4.0-0.editable-py3-none-any.whl size=7089 sha256=6ea93a7f6f8ae1def39132c6945c6af07858a34a893d84db95f132cf02c00eb7
  Stored in directory: /private/var/folders/yg/jdjv7zbj31n9xpnvck_g0zgw0000gn/T/pip-ephem-wheel-cache-e2211u07/wheels/8d/9b/9f/1e8f017a6c3305616d3b9c557f991c4c0971918b9cd79e3a72
Successfully built pyperiscope
  Attempting uninstall: pyperiscope
    Found existing installation: pyperiscope 0.4.0
    Uninstalling pyperiscope-0.4.0:
      Successfully uninstalled pyperiscope-0.4.0


## Run Local Tests

<!-- 
Python code:
import re
from pathlib import Path

def update_minor_version():
    # Update pyproject.toml
    toml_path = Path('pyproject.toml')
    content = toml_path.read_text()
    
    # Find current version
    version_match = re.search(r'version = "(\d+)\.(\d+)\.(\d+)"', content)
    if version_match:
        major, minor, patch = map(int, version_match.groups())
        new_version = f'{major}.{minor+1}.0'
        new_content = re.sub(r'version = "\d+\.\d+\.\d+"', 
                           f'version = "{new_version}"', 
                           content)
        toml_path.write_text(new_content)
        
    # Update __init__.py
    init_path = Path('src/pyperiscope/__init__.py')
    if init_path.exists():
        init_content = init_path.read_text()
        new_init = re.sub(r'__version__ = "\d+\.\d+\.\d+"',
                         f'__version__ = "{new_version}"',
                         init_content)
        init_path.write_text(new_init)
        
    print(f"Updated version to {new_version}")

update_minor_version()
-->

reinstall the local package with test dependecies

In [ ]:
!pip install ".[test]"

Run all tests to verify functionality.<br>
Tests should cover all major functionality.

In [29]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.12.6, pytest-8.3.4, pluggy-1.5.0 -- C:\Python312\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Virtual Production 2\Documents\git2\Pyperiscope
configfile: pyproject.toml
testpaths: tests
collecting ... collected 2 items

tests/test_calc_win.py::test_scope_creation PASSED
tests/test_scope_creation.py::test_scope_creation PASSED

============================== 2 passed in 9.09s ==============================


## Update Minor Version
Updates version numbers in pyproject.toml and __init__.py files.
Increases the minor version (x.Y.z -> x.(Y+1).z)
Example: 0.1.0 -> 0.2.0

In [40]:
import re
from pathlib import Path

def update_version(major_step=None, minor_step=None):
    """
    Check or update version numbers in both pyproject.toml and __init__.py files.
    If no steps provided, just checks and displays current versions.
    
    Args:
        major_step (int, optional): Number of major version steps to increment
        minor_step (int, optional): Number of minor version steps to increment
    
    Returns:
        str: New version number if updating, or None if checking/error
    """
    # Check pyproject.toml
    toml_path = Path('pyproject.toml')
    if not toml_path.exists():
        print(f"Error: {toml_path} not found")
        return None
        
    content = toml_path.read_text()
    toml_version = None
    
    # Find current version
    version_match = re.search(r'version = "(\d+)\.(\d+)\.(\d+)"', content)
    if version_match:
        major, minor, patch = map(int, version_match.groups())
        toml_version = f"{major}.{minor}.{patch}"
        print(f"pyproject.toml version: {toml_version}")
    else:
        print("Error: Could not find version in pyproject.toml")
        return None
        
    # Check __init__.py
    init_path = Path('src/pyperiscope/__init__.py')
    if not init_path.exists():
        print(f"Error: {init_path} not found")
        return None
        
    init_content = init_path.read_text()
    init_version = None
    
    # Try both possible formats
    patterns = [
        r'__version__\s*=\s*"(\d+\.\d+\.\d+)"',  # Format with __version__ and double quotes
        r"__version__\s*=\s*'(\d+\.\d+\.\d+)'",   # Format with __version__ and single quotes
        r"\*\*version\*\*\s*=\s*'(\d+\.\d+\.\d+)'",  # Format with **version** and single quotes
        r'\*\*version\*\*\s*=\s*"(\d+\.\d+\.\d+)"'   # Format with **version** and double quotes
    ]
    
    for pattern in patterns:
        init_version_match = re.search(pattern, init_content)
        if init_version_match:
            init_version = init_version_match.group(1)
            print(f"__init__.py version: {init_version}")
            break
    
    if init_version is None:
        print("Error: Could not find version in __init__.py")
        return None
        
    # If no steps provided, just check version match
    if major_step is None and minor_step is None:
        if toml_version == init_version:
            print("✓ Versions match!")
            return(toml_version)
        else:
            print("⚠ Warning: Versions do not match!")
        return None
        
    # Calculate new version
    major_step = major_step or 0
    minor_step = minor_step or 0
    
    new_major = major + major_step
    new_minor = minor + minor_step if major_step == 0 else 0  # Reset minor version if major version changes
    new_patch = 0  # Always reset patch version
    
    new_version = f'{new_major}.{new_minor}.{new_patch}'
    print(f"\nUpdating to {new_version}")
    
    # Update pyproject.toml
    new_content = re.sub(r'version = "\d+\.\d+\.\d+"', 
                       f'version = "{new_version}"', 
                       content)
    toml_path.write_text(new_content)
    print(f"Updated pyproject.toml")
    
    # Update __init__.py with matching format
    for pattern in patterns:
        if re.search(pattern, init_content):
            template = pattern.replace(r'(\d+\.\d+\.\d+)', new_version)
            new_init = re.sub(pattern, template, init_content)
            init_path.write_text(new_init)
            print(f"Updated __init__.py")
            break
    
    print(f"Successfully updated version to {new_version} in both files")
    return new_version

In [37]:
update_version(minor_step=1)

Found version 0.2.0 in pyproject.toml
Updating to 0.3.0
Updated pyproject.toml
Current __init__.py content:
__version__ = "0.2.0"
from .layer5 import Scope
Found version 0.2.0 in __init__.py
Updated __init__.py
Successfully updated version to 0.3.0 in both files


'0.3.0'

## Update Major Version
Updates version numbers in pyproject.toml and __init__.py files.
Increases the major version (X.y.z -> (X+1).0.0)
Example: 1.2.3 -> 2.0.0

In [36]:
update_version(major_step=1)

Found version 0.3.0 in pyproject.toml
Updating to 1.0.0
Updated pyproject.toml
Current __init__.py content:
__version__ = "0.3.0"
from .layer5 import Scope
Found version 0.3.0 in __init__.py
Updated __init__.py
Successfully updated version to 1.0.0 in both files


'1.0.0'

## Test PyPI Upload and Install
Upload to Test PyPI first to verify release process.
Then install from Test PyPI to verify it works for users.

In [ ]:
# Build and upload to Test PyPI
!python -m build
!twine upload --repository testpypi dist/*

# Test install from Test PyPI
!pip install --index-url https://test.pypi.org/simple/ pyperiscope

## PyPI Release
Final release to production PyPI.
Make sure all tests pass and Test PyPI install worked.
- [ ] use keyring to handle keys

In [ ]:
!python -m build
!twine upload dist/*

## GitHub Push
Push the release to GitHub.
Update version numbers in code before pushing.

In [43]:
import subprocess

def run_git_command(command):
    """
    Run a git command and return its output.
    
    Args:
        command (list): Command and arguments as list
    
    Returns:
        tuple: (success boolean, output string)
    """
    try:
        result = subprocess.run(
            command,
            check=True,
            capture_output=True,
            text=True
        )
        return True, result.stdout
    except subprocess.CalledProcessError as e:
        return False, e.stderr

def updategit():
    current_version = update_version()
    if current_version:
        # Git add
        success, output = run_git_command(['git', 'add', '.'])
        if not success:
            print(f"Error during git add: {output}")
            return
    
        # Git commit
        commit_msg = f"Release version {current_version}"
        success, output = run_git_command(['git', 'commit', '-m', commit_msg])
        if not success:
            print(f"Error during git commit: {output}")
            return
        print(f"Committed with message: {commit_msg}")
    
        # Git push
        success, output = run_git_command(['git', 'push', 'origin', 'main'])
        if not success:
            print(f"Error during git push: {output}")
            return
        print("Successfully pushed to origin main")

updategit()

pyproject.toml version: 0.3.0
__init__.py version: 0.3.0
✓ Versions match!
Committed with message: Release version 0.3.0
Successfully pushed to origin main
